In [1]:

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from absl import app
from absl import flags

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
import tensorflow_lattice as tfl

In [2]:
# Import helpers
import import_ipynb
from common import buildDatasetForLattice
from common import scaleVolume
from common import extractLatticeWeights
from common import dropColumns
from common import filterBad
from common import splitDataset
from common import normDataset
from common import evaluatePerf
from common import evaluateCustom
from common import extractXGWeights

importing Jupyter notebook from common.ipynb


In [3]:
dataset, columns = buildDatasetForLattice()
dataset = filterBad(dataset)
train_dataset, test_dataset, train_labels, test_labels = splitDataset(dataset, 0.2)
train_stats = train_dataset.describe().transpose()

Rejected 441036 points (0.040460%)


In [7]:
def buildLattice(num_keypoints, lattice_size, columns):
    # Use ParallelCombination helper layer to group togehter calibration layers
    # which have to be executed in paralel in order to be able to use Sequential
    # model. Alternatively you can use functional API.
    combined_calibrators = tfl.layers.ParallelCombination()

    # Configure calibration layers for every feature:

    for ft in columns:
        calibrator = tfl.layers.PWLCalibration(
            # Every PWLCalibration layer must have keypoints of piecewise linear
            # function specified. Easiest way to specify them is to uniformly cover
            # entire input range by using numpy.linspace().
            input_keypoints=np.linspace(dataset[ft].min(),
                                      dataset[ft].max(),
                                      num=num_keypoints),
        # You need to ensure that input keypoints have same dtype as layer input.
        # You can do it by setting dtype here or by providing keypoints in such
        # format which will be converted to deisred tf.dtype by default.
        dtype=tf.float32,
        # Output range must correspond to expected lattice input range.
        output_min=0.0,
        output_max=lattice_size - 1.0,
        monotonicity='increasing')
        combined_calibrators.append(calibrator)

    # Create Lattice layer to nonlineary fuse output of calibrators. Don't forget
    # to specify monotonicity 'increasing' for any dimension which calibrator is
    # monotonic regardless of monotonicity direction of calibrator. This includes
    # partial monotonicity of CategoricalCalibration layer.
    lattice = tfl.layers.Lattice(
      lattice_sizes=[lattice_size for _ in columns],
      monotonicities=['increasing' if (ft == 'rising_idx' or ft == 'falling_idx') else 'none' for x in columns],
      output_min=dataset['delay'].min(),
      output_max=dataset['delay'].max())

    model = keras.models.Sequential()
    # We have just 2 layer as far as Sequential model is concerned.
    # PWLConcatenate layer takes care of grouping calibrators.
    model.add(combined_calibrators)
    model.add(lattice)
    model.compile(loss=keras.losses.mean_squared_error,
                optimizer=keras.optimizers.Adagrad(learning_rate=30.0))
    return model




In [8]:
def trainLattice(model, cols, epochs):
    
    features = train_dataset[cols].values.astype(np.float32)
    target = train_labels.values.astype(np.float32)
    print(features)

    model.fit(features,
            target,
            batch_size=32,
            epochs=epochs,
            validation_split=0.05,
            shuffle=False)
    return model

In [9]:
show_perf = True
train_model = True

Hypercube_8_4 = buildLattice(8,4,columns[:-1])
if (train_model): Hypercube_8_4 = trainLattice(Hypercube_8_4,columns[:-1], 5) #train
else: 
    Hypercube_8_4.build((None,5,))
    Hypercube_8_4.load_weights('./Hypercube_8_4') #load
Hypercube_8_4.save('./Hypercube_8_4')
preds = Hypercube_8_4.predict(train_dataset.values.astype(np.float32))
if (show_perf): evaluatePerf(test_labels, preds)
print('--------------------------------')
extractLatticeWeights(Hypercube_8_4, 'Hypercube_8_4')
print('--------------------------------')


[[  12.  190.  379.   13.    0.]
 [  36.  248.  471. 1943.    0.]
 [  25.   40.  201.   25.    0.]
 ...
 [  26.  135.  150. 2659.    0.]
 [  25.   44.   81. 1828.    0.]
 [  69.  490.  745.    0.    0.]]
Train on 7949276 samples, validate on 418383 samples
Epoch 1/5
7949276/7949276 [==============================] - 1765s 222us/sample - loss: 493988.7786 - val_loss: 36617.0639
Epoch 2/5
7949276/7949276 [==============================] - 1786s 225us/sample - loss: 34636.9126 - val_loss: 25601.3926
Epoch 3/5
7949276/7949276 [==============================] - 1796s 226us/sample - loss: 27404.9112 - val_loss: 22528.6968
Epoch 4/5
7949276/7949276 [==============================] - 1859s 234us/sample - loss: 24732.9986 - val_loss: 21035.4034
Epoch 5/5
7949276/7949276 [==============================] - 1623s 204us/sample - loss: 23080.7143 - val_loss: 20088.8377


ValueError: Found input variables with inconsistent numbers of samples: [2091915, 8367659]

In [88]:
# print(Hypercube_8_2)
# print(train_dataset.values)
# print(train_labels.values)
test = buildLattice(2,8,columns[:-1])
print(test)
# print(Hypercube_8_2.summary())

In [96]:
features = train_dataset[columns[:-1]].values.astype(np.float32)[0:10]
print(features)
target = train_labels.values.astype(np.float32)[0:10]

test.fit(features,
        target,
        batch_size=32,
        epochs=1,
        validation_split=0.05,
        shuffle=False)

[[1.200e+01 1.900e+02 3.790e+02 1.300e+01 0.000e+00]
 [3.600e+01 2.480e+02 4.710e+02 1.943e+03 0.000e+00]
 [2.500e+01 4.000e+01 2.010e+02 2.500e+01 0.000e+00]
 [9.600e+01 1.390e+02 3.180e+02 1.000e+00 0.000e+00]
 [6.600e+01 4.000e+01 1.470e+02 0.000e+00 0.000e+00]
 [2.400e+01 1.470e+02 3.270e+02 1.850e+02 0.000e+00]
 [1.400e+01 1.550e+02 3.370e+02 4.890e+02 0.000e+00]
 [1.050e+02 1.850e+02 3.730e+02 1.200e+01 0.000e+00]
 [4.900e+01 7.000e+01 8.300e+01 3.660e+03 0.000e+00]
 [2.600e+01 5.200e+01 2.150e+02 5.900e+01 0.000e+00]]
Train on 9 samples, validate on 1 samples
9/9 [==============================] - 1s 137ms/sample - loss: 1308289.0000 - val_loss: 2232724.7500


In [98]:
print(test.summary())
preds = test.predict(features)

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
parallel_combination_21 (Par multiple                  40        
_________________________________________________________________
lattice_21 (Lattice)         multiple                  32        
Total params: 72
Trainable params: 72
Non-trainable params: 0
_________________________________________________________________
None


In [99]:
print(preds)

[[ -84.66176]
 [ 204.66684]
 [ -94.94145]
 [-149.71718]
 [-133.84915]
 [ -71.80316]
 [ -34.44699]
 [-163.96371]
 [ 572.02185]
 [ -91.2105 ]]


In [100]:
test.save('./test')

In [103]:
test2 = buildLattice(2,8,columns[:-1])
test2.build((None,5,))
test2.load_weights('./test')

In [104]:
print(test2.predict(features))

[[ -84.66176]
 [ 204.66684]
 [ -94.94145]
 [-149.71718]
 [-133.84915]
 [ -71.80316]
 [ -34.44699]
 [-163.96371]
 [ 572.02185]
 [ -91.2105 ]]


In [85]:
preds = Hypercube_8_2.predict(test_dataset.values.astype(np.float32))
if (show_perf): evaluatePerf(test_labels, preds)

IndexError: list index out of range